In [2]:
import os
import pandas as pd
import numpy as np
from rdkit import Chem
from mordred import Calculator, descriptors
from sklearn.feature_selection import VarianceThreshold
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegressionCV
from sklearn.metrics import f1_score
from sklearn.metrics import fbeta_score
from sklearn.metrics import balanced_accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from joblib import dump, load
import seaborn as sns
from matplotlib import pyplot
from random import randint

In [3]:
class Target:
    
    # path: the path to ligands and decoys .ism files
    def __init__ (self, path):
        self.path=path
        
    calc = Calculator(descriptors, ignore_3D=True)
    
    def ligand_batch_size (self):
        with open (f'{self.path}actives_final.ism', 'r') as f :
            return len(f.readlines ())
    
    def actives (self) :
        with open (f'{self.path}actives_final.ism', 'r') as f :
            actives = list(map(lambda x : x.split (' ')[0], f.readlines ()))
        active_mols = list(map(Chem.MolFromSmiles,actives))
        active_df = self.calc.pandas(active_mols, display=False)
        active_df['Active']=1
        print (f"Actives done")
        return active_df
    
    # Decoys dataframe is constructed with size of three-time as size of actives
    # to avoid imbalancity
    
    def decoys (self) :
        with open (f'{self.path}decoys_final.ism', 'r') as f :
            decoys = list(map(lambda x : x.split (' ')[0], f.readlines ()))
        rand = randint(0,len(decoys)//2)
        decoys_mols = list(map(Chem.MolFromSmiles,decoys[rand:rand+(self.ligand_batch_size()*3)]))
        decoys_df = self.calc.pandas(decoys_mols)
        decoys_df['Active']=0
        print (f"Decoys done")
        return decoys_df
    
    def all_mols (self):
        df=pd.concat([self.actives(),self.decoys()], axis=0)
        df.to_csv(f"{self.path.split('/')[-2]}.csv", index=False)
        print(f"{self.path.split('/')[-1]} Done!")
        return df

In [26]:
class Model:
    def __init__(self, name, path=os.getcwd()):
        self.name=name
        self.df=pd.read_csv(f"{path}/{name}.csv", low_memory=False).select_dtypes(np.number)
        self.current_df=0
        self.clf=0
        self.results=0
    
    # Feature Selection based on Corelation and Variance
    # correlation_threshold : int
    # variance_threshold : float
    
    def feature_selection(self,correlation_threshold=150, variance_threshold=0.005):
        sel = VarianceThreshold(0.2)
        final_selected = pd.concat([self.df.iloc[:,:-1].loc[:, sel.fit(self.df.iloc[:,:-1]).get_support()],self.df.iloc[:,-1:]], axis=1)
        print ("variance-based selection done!")
        corrmat = final_selected.corr().abs().unstack()
        print ("matrix of correlations constructed!")
        corrmat = corrmat[corrmat >= 0.8]
        corrmat = corrmat[corrmat < 1]
        corrmat = pd.DataFrame(corrmat).reset_index()
        corrmat.columns = ['feature1', 'feature2', 'corr']
        corellated_features=corrmat["feature1"].tolist()
        frq={}
        for i in range (len(corellated_features)):
            if corellated_features[i] not in frq:
                frq[corellated_features[i]]=1
            else:
                frq[corellated_features[i]]+=1
        highly_correlated=[k for k in sorted(frq, key=frq.get, reverse=True) if frq[k]>correlation_threshold]
        final_selected.drop(columns=highly_correlated)
        self.current_df = final_selected
        print ('features selected!')
        return final_selected
    
    # Scaling preprocess.
    # using StandardScaler
    
    def scale (self):
        scaling_df = self.feature_selection()
        scaler = StandardScaler()
        scaling_df.iloc[:,1:-1] = pd.DataFrame(scaler.fit_transform(scaling_df.iloc[:,1:-1].values),
                                              columns=scaling_df.iloc[:,1:-1].columns,
                                              index=scaling_df.iloc[:,1:-1].index)
        self.current_df = scaling_df
        print ("scaled!")
        return scaling_df
    
    # Building the model
    
    def modeler(self, save_path=os.getcwd()):
        df = self.scale()
        X = df.iloc[:,:-1]
        y = df.iloc[:,-1]
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=0)
        clf = LogisticRegressionCV(cv=5, Cs=[0.5, 1, 5, 10], random_state=0,
                                   penalty = 'elasticnet', solver='saga', max_iter=5000,
                                   l1_ratios=[0.25,0.5,0.75], n_jobs=-1)
        clf.fit(X_train, y_train)
        self.clf=clf
        print ("model trained!")
        y_pred = clf.predict (X_test)
        y_true = y_test
        f1 = f1_score(y_true, y_pred)
        fb = fbeta_score(y_true, y_pred, beta = 0.5)
        balanced_accuracy = balanced_accuracy_score(y_true, y_pred)
        filename = f"{save_path}/{self.name}.sav"
        dump(clf, filename)
        test_results=list(zip(df.columns,clf.coef_[0])) + [("F1 score",f1) + ("F-Beta score", fb) +
                                                     ("Balanced Accuracy score", balanced_accuracy)]
        self.results=test_results
        return test_results

In [42]:
# calculating ratio of highly correlated features for each target
def corr_mapper (x):
    a=0
    if x>=0.5 and x!=1:
        a=1
    elif x==1:
        a=0
    return a
corr_matrix = [pd.read_csv(f"dfs/{tar}",low_memory=False).select_dtypes(np.number).corr() for tar in dfs]
corr_mapped=[np.vectorize(corr_mapper)(corr_matrix_tar[:]) for corr_matrix_tar in corr_matrix]
high_corrs=[np.sum(corr_mapped_tar)/corr_mapped_tar.size for corr_mapped_tar in corr_mapped]

In [19]:
dfs = os.listdir("dfs")
names = [tar.split(".")[0] for tar in dfs]

In [28]:
print (f"{names[0]}")
mmp13 = Model(f"{names[0]}","dfs")
mmp13_scores = mmp13.modeler("dfs")
print (mmp13_scores[-1])

('F1 score', 0.9868421052631579, 'F-Beta score', 0.9907529722589169, 'Balanced Accuracy score', 0.9890027610089381)


In [31]:
print (f"{names[1]}")
fa10 = Model(f"{names[1]}","dfs")
fa10_scores = fa10.modeler("dfs")
print (fa10_scores[-1])

fa10
variance-based selection done!
matrix of correlations constructed!
features selected!
scaled!
model trained!
('F1 score', 0.993006993006993, 'F-Beta score', 0.9930069930069931, 'Balanced Accuracy score', 0.9952344609705016)


In [36]:
print (f"{names[2]}")
dpp4 = Model(f"{names[2]}","dfs")
dpp4_scores = dpp4.modeler("dfs")
print (dpp4_scores[-1])

dpp4
variance-based selection done!
matrix of correlations constructed!
features selected!
scaled!
model trained!
('F1 score', 0.9964412811387899, 'F-Beta score', 0.9943181818181818, 'Balanced Accuracy score', 0.9987277353689568)


In [38]:
print (f"{names[3]}")
mk14 = Model(f"{names[3]}","dfs")
mk14_scores = mk14.modeler("dfs")
print (mk14_scores[-1])

mk14
variance-based selection done!
matrix of correlations constructed!
features selected!
scaled!
model trained!
('F1 score', 0.9770491803278689, 'F-Beta score', 0.9675324675324676, 'Balanced Accuracy score', 0.9896573208722741)
